In [12]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '8,9'
import torch
import torch.nn.functional as F
from safetensors.torch import load_file
import json
from tqdm import tqdm
import dei_utils as Dei

# model.model.layers[0].block_sparse_moe.gate.weight.data
# model.model.layers[0].block_sparse_moe.experts[0].w1??
# model.model.layers[0].block_sparse_moe.experts[0].w1.weight.data.shape

In [18]:
def get_expert(layer):
    json_file_path = "/new_data/yanghq/models/mistralai/Mixtral-8x7B-Instruct-v0.1/model.safetensors.index.json"
    with open(json_file_path, 'r') as f:
        data = json.load(f)
        data=data["weight_map"]
    shard_list=[]
    for i in range(8):
        for j in range(3):
            weight_idx = f'model.layers.{layer}.block_sparse_moe.experts.{i}.w{j+1}.weight'
            shard_path = data[weight_idx]
            if shard_path not in shard_list:
                shard_list.append(shard_path)
    weight_dict={}
    for shard_path in shard_list:
        shard_path = f'/new_data/yanghq/models/mistralai/Mixtral-8x7B-v0.1/{shard_path}'
        shard_weights = load_file(shard_path)
        for i in range(8):
            for j in range(3):
                weight_idx = f'model.layers.{layer}.block_sparse_moe.experts.{i}.w{j+1}.weight'
                if weight_idx in shard_weights.keys():
                    weight_dict.update({weight_idx:shard_weights[weight_idx]})
    return weight_dict
    assert len(weight_dict) == 24
    weight_list=[]
    for w in weight_dict.values():
        if w.shape[0] != 4096:
            w=w.transpose(0,1)
        weight_list.append(w)
    weights=torch.stack(weight_list)
    weights = weights.reshape([8,3,4096,-1])
    # weights = weights.reshape([8,-1])
    return weights
a=get_expert(0)


In [25]:
for i in a.values():
    print(i.shape)

torch.Size([14336, 4096])
torch.Size([4096, 14336])
torch.Size([14336, 4096])
torch.Size([14336, 4096])
torch.Size([4096, 14336])
torch.Size([14336, 4096])
torch.Size([14336, 4096])
torch.Size([4096, 14336])
torch.Size([14336, 4096])
torch.Size([14336, 4096])
torch.Size([4096, 14336])
torch.Size([14336, 4096])
torch.Size([14336, 4096])
torch.Size([4096, 14336])
torch.Size([14336, 4096])
torch.Size([14336, 4096])
torch.Size([4096, 14336])
torch.Size([14336, 4096])
torch.Size([14336, 4096])
torch.Size([4096, 14336])
torch.Size([14336, 4096])
torch.Size([14336, 4096])
torch.Size([4096, 14336])
torch.Size([14336, 4096])


In [3]:
def get_router(layer):
    json_file_path = "/new_data/yanghq/models/mistralai/Mixtral-8x7B-Instruct-v0.1/model.safetensors.index.json"
    with open(json_file_path, 'r') as f:
        data = json.load(f)
    data=data["weight_map"]
    weight_idx = f'model.layers.{layer}.block_sparse_moe.gate.weight'
    shard_path = data[weight_idx]
    shard_path = f'/new_data/yanghq/models/mistralai/Mixtral-8x7B-v0.1/{shard_path}'
    shard_weights = load_file(shard_path)
    weights = shard_weights[weight_idx]
    return weights


In [ ]:
cos_sim = torch.zeros(32, 8, 8)
for l in tqdm(range(32)):
    data = get_router(l).to(torch.double)
    for i in range(8):
        for j in range(i+1):
            cos_sim[l, i, j] = F.cosine_similarity(data[i].unsqueeze(0), data[j].unsqueeze(0))
            cos_sim[l, j, i] = cos_sim[l, i, j]
Dei.save(cos_sim, 'dist/rou_cos')

cos_sim = torch.zeros(32, 8, 8)
for l in tqdm(range(32)):
    torch.cuda.empty_cache()
    data = get_expert(l).to(torch.double).to('cuda:0')
    for i in range(8):
        for j in range(i+1):
            cos_sim[l, i, j] = F.cosine_similarity(data[i].unsqueeze(0), data[j].unsqueeze(0))
            cos_sim[l, j, i] = cos_sim[l, i, j]
Dei.save(cos_sim, 'dist/exp_cos')


  0%|          | 0/32 [00:00<?, ?it/s]

100%|██████████| 32/32 [00:00<00:00, 188.77it/s]


Saving tensor to /new_data/yanghq/data/dist/rou_cos.pt


 41%|████      | 13/32 [01:15<01:46,  5.59s/it]

In [4]:
cos_sim = torch.zeros(32, 8, 8)
for l in tqdm(range(32)):
    data = get_router(l).to(torch.double)
    for i in range(8):
        for j in range(i+1):
            diff = data[i] - data[j]
            cos_sim[l, i, j] = torch.sum(diff ** 2) / diff.numel()
            cos_sim[l, j, i] = cos_sim[l, i, j]
Dei.save(cos_sim, 'dist/rou_mse')

cos_sim = torch.zeros(32, 8, 8)
for l in tqdm(range(32)):
    torch.cuda.empty_cache()
    data = get_expert(l).to(torch.double).to('cuda:0')
    for i in range(8):
        for j in range(i+1):
            diff = data[i] - data[j]
            cos_sim[l, i, j] = torch.sum(diff ** 2) / diff.numel()
            cos_sim[l, j, i] = cos_sim[l, i, j]
Dei.save(cos_sim, 'dist/exp_mse')


100%|██████████| 32/32 [00:00<00:00, 155.50it/s]


Saving tensor to /new_data/yanghq/data/dist/rou_mse.pt


100%|██████████| 32/32 [02:58<00:00,  5.59s/it]

Saving tensor to /new_data/yanghq/data/dist/exp_mse.pt
